### Librairie pour le modèle WCGAN-GP

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import Dropout 
from tensorflow.keras import backend
from tensorflow.keras.initializers import HeNormal,GlorotUniform
import numpy as np
import pandas as pd
from traitement_image import image_processing
from utils import generator_withCNN_loss, wasserstein_loss, generator_loss, generate_batches_cond, generate_cond, Conv2DCircularPadding, load_parquet_files_cond_red, newTorchSign, transform_batch
import matplotlib.pyplot as plt
import os

### Librairie pour le modèle CNN-RNN 

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split, Dataset, ConcatDataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.transforms as transforms
from torchvision.io import read_image
from torchmetrics.regression import WeightedMeanAbsolutePercentageError, MeanAbsoluteError
from e2cnn import gspaces
from e2cnn import nn as enn
import random

# Recupération du modèle CNN-RNN 

In [ ]:
# Définir le modèle CNN-RNN
#On va utiliser des convolutions équivariantes aux symmétries et aux rotations à l'aide du package e2cnn.

class CNNLSTM(nn.Module):
    def __init__(self):
        super(CNNLSTM, self).__init__()
        self.g_space = gspaces.FlipRot2dOnR2(N=4) #Définition de l'espace invariant aux symmétries horizontales, verticales
                                                  #et aux rotations de 90°
        
        in_type = enn.FieldType(self.g_space, [self.g_space.trivial_repr])
        self.input_type = in_type

        #Pour chaque couche de convolution équivariante, 8 sous-couches de convolutions parallèles sont en fait créés
        #(une pour chaque combinaison de symmétrie/rotation possible). 
        #Plus besoin de donner des les images en rotation ou en symmétrie des microstructures en entraînement.
        
        self.conv1 = self._get_equivariant_conv(1,16)
        self.conv2 = self._get_equivariant_conv(16, 32)
        self.conv3 = self._get_equivariant_conv(32, 48)
        self.conv4 = self._get_equivariant_conv(48, 64)
        self.conv5_1 = self._get_equivariant_conv(64, 82)
        self.conv5_2 = self._get_equivariant_conv(64, 82)
        self.conv6_1 = self._get_equivariant_conv(82, 112)

        #A la fin des couches de convolution équivariantes, on applique un "Group Pooling" pour grouper l'ensemble des caractéristiques
        #trouvées par chaque combinaison de rotation / symmétrie afin d'en extraire des caractéristiques intrinsèques à la microstructure,
        #indépendamment de sa rotation/symmétrie.
        
        self.gpool_1 = enn.GroupPooling(enn.FieldType(self.g_space, [self.g_space.regular_repr]*112))
        self.gpool_2 = enn.GroupPooling(enn.FieldType(self.g_space, [self.g_space.regular_repr]*82))
        
        self.flatten = nn.Flatten()
        self.fc_1 = nn.Linear(360, 30)
        self.fc_2 = nn.Linear(360, 30)
        self.GRU_1 = nn.GRU(4*4*112, 360, num_layers = 3, dropout = 0.2) #Pour le module d'élasticité
        self.lstm_2 = nn.LSTM(8*8*82, 360, num_layers = 2, dropout = 0.3)  #Pour le module de perte
        
        
    def _get_equivariant_conv(self, in_channels, out_channels, kernel_size=3):
        # Définir in_type selon le nombre de canaux d'entrée
        if in_channels == 1:
            in_type = enn.FieldType(self.g_space, [self.g_space.trivial_repr] * in_channels)
        else:
            in_type = enn.FieldType(self.g_space, [self.g_space.regular_repr] * in_channels)
        
        # Définir out_type (toujours regular_repr pour la sortie)
        out_type = enn.FieldType(self.g_space, [self.g_space.regular_repr] * out_channels)
        
        # Créer le bloc de convolution
        block = enn.SequentialModule(
            enn.R2Conv(in_type, out_type, kernel_size=kernel_size, padding=1, padding_mode="circular"),
            enn.InnerBatchNorm(out_type),
            enn.ReLU(out_type, inplace=True),
            enn.PointwiseMaxPoolAntialiased(out_type, kernel_size=2)
        )
        return block


    def forward(self, x):
        #Partie commune CNN
        
        x = enn.GeometricTensor(x, self.input_type)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        #Décomposition en 2 sous-blocs CNN+RNN parallèles:
        
        #1er bloc:
        x1 = self.conv5_1(x)
        x1 = self.conv6_1(x1)
        x1 = self.gpool_1(x1)
        x1 = x1.tensor
        x1 = self.flatten(x1)
        x1, h_1 = self.GRU_1(x1)
        x1 = (self.fc_1(x1))
        
        #2nd bloc (moins complexe que le premier):
        x2 = self.conv5_2(x)
        x2 = self.gpool_2(x2)
        x2 = x2.tensor
        x2 = self.flatten(x2)
        x2, h_2 = self.lstm_2(x2)
        x2 = (self.fc_2(x2))
        return x1,x2

In [ ]:
load_path = "/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/projet-dunne-main/Codes/best_model_1.pt" #Endroit où le modèle a été sauvegardé
modelCNN = CNNLSTM()
modelCNN.load_state_dict(torch.load(load_path,map_location=torch.device('cpu')))
modelCNN.eval()

# Définir la fonction de perte
criterion = WeightedMeanAbsolutePercentageError()

# Generator

In [ ]:
generatorGP = tf.keras.models.load_model('/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_InesHafassaMaiza/model/generateur_red_gp.keras')

# Discriminator

In [ ]:
discriminatorGP = tf.keras.models.load_model('/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_InesHafassaMaiza/model/discriminateur_red_gp.keras')

# WCGAN

In [ ]:
class GAN(models.Model):
    def __init__(self, generator, discriminator, modelCNN ,latent_dim, batch_size):
        """
        Définition du modèle complet générateur + discriminateur
        :param generator: le générateur
        :param discriminator: le disriminateur
        :param data_path: Le chemin d'accès aux données prétraitées
        """
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.modelCNN = modelCNN
        self.compile_discriminator()
        self.compile_gan()
        self.latent_dim = latent_dim
        self.cond_dim = 62
        self.image_dim = (128,128)
        self.batch_size = batch_size
        # Définir les optimizers ICI, pas dans les sous-modèles
        self.generator_optimizer = RMSprop(learning_rate=0.005)
        self.discriminator_optimizer = RMSprop(learning_rate=0.00005)
        
        # Compiler le discriminateur
        self.discriminator.compile(
            optimizer=self.discriminator_optimizer,
            loss=self.gradient_penalty_loss
        )

    def gradient_penalty_loss(self, real_images, fake_images, real_sign):
        """
        Calcul du gradient penalty pour un WGAN-GP.
        :param real_images: Données réelles.
        :param fake_images: Données générées.
        :param real_sign: Signature réelle.
        :return: Perte de gradient penalty.
        """
        lambda_gp = 10
        
        # Interpolation entre les échantillons réels et générés
        alpha = tf.random.uniform((self.batch_size, 1, 1, 1), minval=0.0, maxval=1.0)
        interpolated = (alpha * real_images) + ((1 - alpha) * fake_images)
        #interpolated = np.array(interpolated)
        real_sign = tf.convert_to_tensor(real_sign)
        
        # Calcul des prédictions du discriminateur sur les échantillons interpolés
        with tf.GradientTape() as tape:
            tape.watch(interpolated)  # Surveiller interpolated pour le calcul des gradients
            pred = self.discriminator([interpolated, real_sign])
                                                     #, training=True
                                                     
        
        # Calcul des gradients
        gradients = tape.gradient(pred, interpolated)
        
        # Calcul de la norme des gradients
        gradients_sqr = tf.square(gradients)
        gradients_sqr_sum = tf.reduce_sum(gradients_sqr, axis=[1, 2, 3])
        gradient_l2_norm = tf.sqrt(gradients_sqr_sum)
        
        # Calcul du gradient penalty
        gradient_penalty = tf.reduce_mean(tf.square(gradient_l2_norm - 1.0))
        
        # Perte totale
        return lambda_gp * gradient_penalty
            
        
    def compile_discriminator(self):
        self.discriminator.compile(loss=self.gradient_penalty_loss, optimizer=RMSprop(learning_rate=0.00005), metrics=['accuracy'])
        #self.discriminator.trainable = False

    # def compile_gan(self):
    #     latent = layers.Input(shape=(self.generator.latent_dim,))
    #     cond = layers.Input(shape=(self.generator.cond_dim,))
    #     fake_image = self.generator([latent, cond])
    #     input_disc = [fake_image, cond]
    #     validity = self.discriminator(input_disc)

    #     self.model = models.Model([latent,cond], validity)
    #     self.model.compile(loss=generator_loss, optimizer=RMSprop(learning_rate=0.005))

    def compile_gan(self):
        # Initialiser les optimizers
        self.generator_optimizer = RMSprop(learning_rate=0.005)
        self.discriminator_optimizer = RMSprop(learning_rate=0.00005)
        
        # Compiler le discriminateur (Wasserstein + GP)
        self.discriminator.compile(
            optimizer=self.discriminator_optimizer,
            loss=self.gradient_penalty_loss
        )
        
        # Le générateur nécessite un entraînement personnalisé (via un pas manuel)
        self.discriminator.trainable = False  # Gelé pendant l'entraînement du générateur

    def generate_latent_points(self, latent_dim, n_samples):
        """
        Génère n_samples vecteurs latents
        :param latent_dim: dimension de l'espace latent
        :param n_samples: taille de l'échantillon crée
        :return:
        """
        x_input = np.random.randn(latent_dim * n_samples)
        x_input = x_input.reshape(n_samples, latent_dim)
        return x_input

    def generate_cond(self, cond):
        """
        Mise en forme de la condition avant passage dans le modèle
        :param cond: Liste des conditions du batch correspondant lors de l'entraînement
        :return: les conditions mises en forme
        """
        x_input = pd.concat(cond, axis=1)
        x_input = x_input.T
        x_input = np.array(x_input)
        return x_input
    

    def generate_real_samples(self, n_samples):
        """
        
        Charge un batch d'images réelles
        :param n_samples: taille du batch
        :return: Batch d'images réelles
        """
        dfs = self.data
        dfs_array = [(df[0].to_numpy(), df[0].to_numpy()) for df in dfs] ### /!\
        # np.random.shuffle(dfs_array)

        sampled_indices = np.random.choice(len(dfs_array), size=n_samples, replace=False)

        # Sélectionner les arrays échantillonnés
        real_samples = [dfs_array[i] for i in sampled_indices]
        real_samples = np.stack(real_samples, axis=0)
        real_samples = np.expand_dims(real_samples, axis=-1)
        labels = -(np.ones((n_samples, 1))) # Création des labels associés aux images réelles

        return real_samples, labels
    



### Fonctions (plot, save)

In [ ]:

def plot_training_history(d_losses, g_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label='Discriminator Loss', linestyle='--')
    plt.plot(g_losses, label='Generator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('WGAN Training History')
    plt.show()

def generate_and_save_outputs(root_folder, gan, latent_dim):
    """
    CHarge des signatures de root_folder puis génère les structures associées selon le gan
    :param root_folder:
    :param gan:
    :param latent_dim:
    :return:
    """
    # Parcourir tous les sous-dossiers
    for folder in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder)
        if os.path.isdir(folder_path):
            condition_file = os.path.join(folder_path, 'craft_s12.parquet')

            # Vérifier si le fichier de condition existe
            if os.path.exists(condition_file):
                # Charger le DataFrame de condition
                condition = pd.read_parquet(condition_file)
                condition = condition.T
                # Générer un vecteur latent aléatoire
                latent_vector = gan.generate_latent_points(latent_dim, 1)

                # Générer une sortie du GAN
                generated_output = gan.generator.predict([latent_vector, condition])

                # Sauvegarder la sortie dans le même dossier
                output_file = os.path.join(folder_path, 'generated_output_red.npy')
                np.save(output_file, generated_output)

## Training WCGAN-GP

In [ ]:
def train_unrolled_wgan_gp(generator, discriminator, gan, data, latent_dim, 
                           n_epochs, n_critic, batch_size, K_unroll=5):
    """
    Entraînement d'un Unrolled WGAN-GP
    :param generator: le générateur
    :param discriminator: le discriminateur
    :param gan: le modèle GAN complet
    :param data: données d'entraînement
    :param latent_dim: dimension de l'espace latent
    :param n_epochs: nombre d'époques
    :param n_critic: nombre de mises à jour du discriminateur par mise à jour du générateur
    :param batch_size: taille des batchs
    :param K_unroll: nombre de pas de déroulement du discriminateur
    """
    d_losses, g_losses = [], []
    current_epoch = 0

    gan.discriminator.compile(loss=gan.gradient_penalty_loss, optimizer=RMSprop(learning_rate=0.00005), metrics=['accuracy'])

    for epoch in range(n_epochs):
        batches = generate_batches_cond(data, batch_size)
        current_epoch += 1
    


        for batch in batches:
            # Mettre à jour le discriminateur plusieurs fois
            for _ in range(n_critic):
                print("epoch",epoch)
                y_real = -np.ones((batch_size, 1))  # Labels réels (-1)
                
                voxel_data = [np.expand_dims(sample[0], axis=-1) for sample in batch]
                signature_data = [sample[1].to_numpy() for sample in batch]

                X_real_tab_np = np.array(voxel_data)
                X_real_sign_np = np.array(signature_data)

                latent = gan.generate_latent_points(latent_dim, batch_size)
                real_sign = np.squeeze(X_real_sign_np, axis=-1)
                input_model = layers.Concatenate()([latent, real_sign])
                X_fake_tab_np = gan.generator.predict(input_model)

                # Sauvegarde des poids du discriminateur
                original_disc_weights = discriminator.get_weights()

                discriminator.trainable = True  # Rendre le discriminateur entraînable


                with tf.GradientTape() as disc_tape:
                    pred_real = discriminator([X_real_tab_np, X_real_sign_np], training=True)
                    pred_fake = discriminator([X_fake_tab_np, X_real_sign_np], training=True)
                    wasserstein_loss = tf.reduce_mean(pred_fake) - tf.reduce_mean(pred_real)
                    gp_loss = gan.gradient_penalty_loss(X_real_tab_np, X_fake_tab_np, X_real_sign_np)
                    d_loss = wasserstein_loss + gp_loss

                gradients = disc_tape.gradient(d_loss, discriminator.trainable_variables)
                print(f"Discriminator Gradients: {[g.numpy().sum() for g in gradients if g is not None]}")

                gan.discriminator_optimizer.apply_gradients(zip(gradients, discriminator.trainable_variables))

            # Unrolling du Discriminateur
            for _ in range(K_unroll):
                with tf.GradientTape() as unroll_tape:
                    pred_real = discriminator([X_real_tab_np, X_real_sign_np], training=True)
                    pred_fake = discriminator([X_fake_tab_np, X_real_sign_np], training=True)
                    wasserstein_loss = tf.reduce_mean(pred_fake) - tf.reduce_mean(pred_real)
                    gp_loss = gan.gradient_penalty_loss(X_real_tab_np, X_fake_tab_np, X_real_sign_np)
                    d_loss_unroll = wasserstein_loss + gp_loss

                gradients_unroll = unroll_tape.gradient(d_loss_unroll, discriminator.trainable_variables)
                discriminator.set_weights([
                    w - 0.00005 * g for w, g in zip(discriminator.get_weights(), gradients_unroll)
                ])

            # **Mise à jour du générateur**
            latent = gan.generate_latent_points(latent_dim, batch_size)
            cond = [sample[1] for sample in batch]
            cond = gan.generate_cond(cond)
            real_sign = np.expand_dims(cond, axis=-1)
            real_sign = tf.convert_to_tensor(real_sign)

            input_model = layers.Concatenate()([latent, cond])

            with tf.GradientTape() as gen_tape:
                fake_images = gan.generator(input_model, training=True)
                validity = discriminator([fake_images, real_sign], training=False)
                g_loss = generator_withCNN_loss(
                    y_true=None, 
                    y_pred=validity, 
                    real_sign=real_sign, 
                    fake_images=fake_images,
                    CNNmodel=modelCNN,  
                    criterion=criterion,
                    lambda_sign=-10000.0
                )

            gradients = gen_tape.gradient(g_loss, generator.trainable_variables)
            print(f"Generator Gradients: {[g.numpy().sum() for g in gradients if g is not None]}")
            gan.generator_optimizer.apply_gradients(zip(gradients, generator.trainable_variables))

            # Restauration des poids originaux du discriminateur
            discriminator.set_weights(original_disc_weights)

            # Enregistrer les pertes
            d_losses.append(-d_loss)
            g_losses.append(g_loss)
            
        # Sélectionner aléatoirement deux indices de conditions
            random_indices = np.random.choice(cond.shape[0], 2, replace=False)
            conditions_selected = cond[random_indices]
        # Générer des points latents
            latent_points = gan.generate_latent_points(latent_dim, 2)  # Pour deux échantillons

            entree = layers.Concatenate()([latent_points, conditions_selected])
        # Générer des images avec le générateur
            
            generated_images = gan.generator.predict(entree)
            
            print(f"Generated Images Shape: {generated_images.shape}")


        # Afficher les images
            for i in range(2):
                plt.imshow(generated_images[i], cmap='gray')
                plt.show()


        # Display results 
        plot_training_history(d_losses, g_losses)
        print(f"Epoch {current_epoch}, [D loss: {d_losses[-1]}], [G loss: {g_loss}]")
        generator_weights = [layer.get_weights()[0].flatten() for layer in generator.layers if len(layer.get_weights()) > 0]
        discriminator_weights = [layer.get_weights()[0].flatten() for layer in discriminator.layers if len(layer.get_weights()) > 0]
        
        plt.figure(figsize=(10,5))
        for weights in generator_weights :
            plt.hist(weights, bins = 50, alpha = 0.5)
        plt.title('Histogramme des poids du générateur')
        plt.show()
        
        plt.figure(figsize=(10,5))
        for weights in discriminator_weights :
            plt.hist(weights, bins = 50, alpha = 0.5)
        plt.title('Histogramme des poids du discriminateur')
        plt.show()

**Lancement du code**

In [ ]:


#with open("/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_Daviet_Colin/preproc2.py", "r") as file:
    #exec(file.read())


In [ ]:
# Définir le nombre d'époques et la taille du lot
latent_dim = 50
n_epochs = 2
batch_size = 64
n_critic = 1
data_path = '/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_InesHafassaMaiza/DATA2'  # Définir le chemin d'accès aux données
data = load_parquet_files_cond_red(data_path, test = False)
print(f"Taille de `data`: {len(data)}")
if len(data) == 0:
    print("Aucune donnée n'a été chargée.")


# Génération d'images

In [ ]:
batch_size = 20

gan = GAN(generatorGP, discriminatorGP, modelCNN, latent_dim, batch_size)

generatorGP.summary()
discriminatorGP.summary()

In [ ]:
generator_weights = [layer.get_weights()[0].flatten() for layer in generatorGP.layers if len(layer.get_weights()) > 0]
discriminator_weights = [layer.get_weights()[0].flatten() for layer in discriminatorGP.layers if len(layer.get_weights()) > 0]

plt.figure(figsize=(10,5))
for weights in generator_weights :
    plt.hist(weights, bins = 50, alpha = 0.5)
plt.title('Histogramme des poids du générateur')
plt.show()

plt.figure(figsize=(10,5))
for weights in discriminator_weights :
    plt.hist(weights, bins = 50, alpha = 0.5)
plt.title('Histogramme des poids du discriminateur')
plt.show()

# Entraîner le GAN
train_unrolled_wgan_gp(generatorGP, discriminatorGP, gan, data, latent_dim, n_epochs, n_critic, batch_size)

In [ ]:
#generatorGP.save('/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_InesHafassaMaiza/model/fine_tuned_unrolled_generateur_red_gp_with_CNN.keras')
#discriminatorGP.save('/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_InesHafassaMaiza/model/fine_tuned_unrolled_discriminateur_red_gp_with_CNN.keras')

In [ ]:
from tensorflow.keras.models import load_model
latent_dim = 38
n_epochs = 2
batch_size = 64
n_critic = 1
#generatorGP = tf.keras.models.load_model('C:/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/model2/generateur_red_gp2.keras')
generatorGP = tf.keras.models.load_model('/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_InesHafassaMaiza/model/generateur_red_gp.keras')
#discriminatorGP = tf.keras.models.load_model('/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_InesHafassaMaiza/model/fine_tuned_unrolled_discriminateur_red_gp_with_CNN.keras')
#discriminatorGP = tf.keras.models.load_model('C:/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/model2/discriminateur_red_gp2.keras')
discriminatorGP = tf.keras.models.load_model('/Users/Bader/Desktop/Mines 2A/Projet 2A/Ines/projet-dep/Projet_InesHafassaMaiza/model/discriminateur_red_gp.keras')
gan = GAN(generatorGP, discriminatorGP, modelCNN, latent_dim, batch_size)

In [ ]:
import numpy as np
import torch
from utils import transform_batch, newTorchSign
from traitement_image import image_processing
from scipy import stats  # Pour IC à 95%

# === Paramètres ===
n_samples = 10000
latent_dim = 38
batch_size = 12
n_batches = n_samples // batch_size

# === Accumulateurs pour les erreurs ===
all_errors_modulus1 = []
all_errors_modulus2 = []

# === Boucle principale ===
for batch_idx in range(n_batches):
    print(f"Batch {batch_idx+1}/{n_batches}")
    
    # === Génération des données ===
    batches = generate_batches_cond(data, batch_size)
    i = random.randrange(len(batches))
    batch = batches[i]
    
    cond = gan.generate_cond([sample[1] for sample in batch])
    real_sign = np.expand_dims(cond, axis=-1)
    z = gan.generate_latent_points(latent_dim, batch_size)

    input_model = layers.Concatenate()([z, cond])
    generated_samples = generatorGP.predict_on_batch(input_model)

    # === Préparation et prédiction ===
    fake_images_torch = transform_batch(generated_samples)
    pred_sign1, pred_sign2 = modelCNN(fake_images_torch)
    real_sign1, real_sign2 = newTorchSign(real_sign)

    # === Calcul des erreurs ===
    for i in range(batch_size):
        err1 = criterion(real_sign1[i], pred_sign1[i]).item()
        err2 = criterion(real_sign2[i], pred_sign2[i]).item()
        all_errors_modulus1.append(err1)
        all_errors_modulus2.append(err2)

# === Calculs statistiques ===
errors1 = np.array(all_errors_modulus1)
errors2 = np.array(all_errors_modulus2)

# Moyennes
mean1 = np.mean(errors1)
mean2 = np.mean(errors2)

# Écart-types
std1 = np.std(errors1, ddof=1)  # ddof=1 pour échantillon
std2 = np.std(errors2, ddof=1)

# Erreurs-types
stderr1 = std1 / np.sqrt(len(errors1))
stderr2 = std2 / np.sqrt(len(errors2))

# Intervalles de confiance à 95%
ci1 = stats.norm.interval(0.95, loc=mean1, scale=stderr1)
ci2 = stats.norm.interval(0.95, loc=mean2, scale=stderr2)

# === Affichage final ===
print("\n--- Résultats sur {:,} microstructures ---".format(n_samples))
print(f"Storage modulus (MAPE) : {mean1*100:.2f} ± {stderr1*100:.2f} % (IC 95%: {ci1[0]*100:.2f} – {ci1[1]*100:.2f} %)")
print(f"Loss modulus    (MAPE) : {mean2*100:.2f} ± {stderr2*100:.2f} % (IC 95%: {ci2[0]*100:.2f} – {ci2[1]*100:.2f} %)")


In [ ]:
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, distance_transform_edt
from skimage.morphology import disk, binary_erosion
from skimage.segmentation import watershed 
from skimage.filters import threshold_otsu
from skimage.measure import label
import cv2
from e2cnn import gspaces
from e2cnn.nn import GeometricTensor
def image_processing_single(image_fake):
    W, H = 128, 128  # Taille de l’image
    dpi = 25
    fixed_radius = H * 3.98942280401433e-2  # Rayon fixe

    # Étape 1 : conversion en image PIL
    X_fake_reshaped = (image_fake * 255).astype(np.uint8).reshape(H, W)
    image = Image.fromarray(X_fake_reshaped, mode='L') 

    # Conversion OpenCV
    image_cv = np.array(image)
    smoothed = gaussian_filter(image_cv, sigma=0.5)
    tresh_value = threshold_otsu(smoothed)
    binary = (smoothed > tresh_value).astype(np.uint8) * 255
    binary_opened = binary_erosion(binary, disk(2))

    distance = distance_transform_edt(binary_opened)
    markers = label(distance > 0.3 * distance.max())
    local_maxi = watershed(-distance, markers, mask=binary_opened)
    binary_local_maxi = (local_maxi > 0).astype(np.uint8) * 255

    # Étape 2 : création de l'image avec cercles
    gray = binary_local_maxi
    _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    fig, ax = plt.subplots(figsize=(W/dpi, H/dpi), dpi=dpi)
    ax.imshow(np.zeros_like(image_cv), cmap='gray')

    for cnt in contours:
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            ax.add_patch(plt.Circle((cx, cy), fixed_radius, color='white', fill=True, alpha=0.9))
            for dx, dy in [(-W, 0), (W, 0), (0, -H), (0, H), (-W, -H), (-W, H), (W, -H), (W, H)]:
                ax.add_patch(plt.Circle((cx + dx, cy + dy), fixed_radius, color='white', fill=True, alpha=0.9))

    ax.set_position([0, 0, 1, 1])
    ax.set_xticks([]), ax.set_yticks([]), ax.set_frame_on(False), ax.set_facecolor('black')
    fig.canvas.draw()

    image_array = np.array(fig.canvas.renderer.buffer_rgba())
    plt.close(fig)

    gray_image = np.mean(image_array[:H, :W, :3], axis=2)
    binary_array = (gray_image > 100).astype(np.float32)

    binary_array = np.expand_dims(binary_array, axis=0)
    upscaled_array = np.repeat(np.repeat(binary_array, 2, axis=1), 2, axis=2)

    tensor = torch.from_numpy(upscaled_array)
    tensor = torch.where(tensor == 1, torch.tensor(0.5), torch.tensor(-0.5))
    return tensor  # shape (1, 256, 256)


In [ ]:
import numpy as np
import torch
from utils import transform_batch, newTorchSign
from traitement_image import image_processing
import random

# === Paramètres ===
n_samples = 10000
latent_dim = 38
batch_size = 12
n_batches = n_samples // batch_size

# === Accumulateurs pour les erreurs ===
all_errors_modulus1 = []
all_errors_modulus2 = []

# === Boucle principale ===
for batch_idx in range(n_batches):
    print(f"Batch {batch_idx+1}/{n_batches}")
    
    # Génération d'un batch aléatoire
    batches = generate_batches_cond(data, batch_size)
    i = random.randrange(len(batches))
    batch = batches[i]

    # Génération conditionnelle
    cond = gan.generate_cond([sample[1] for sample in batch])
    real_sign = np.expand_dims(cond, axis=-1)
    z = gan.generate_latent_points(latent_dim, batch_size)
    input_model = layers.Concatenate()([z, cond])
    generated_samples = generatorGP.predict_on_batch(input_model)

    # === Traitement d'image et prédiction ===
    for k in range(batch_size):
        image_fake = generated_samples[k]
        
        # Traite une seule image et retourne le tenseur prêt à être passé au CNN
        processed_tensor = image_processing_single(image_fake)  # ⇐ À extraire de ta fonction image_processing
        
        pred_sign1, pred_sign2 = modelCNN(processed_tensor.unsqueeze(0))  # Shape [1, 1, 256, 256]
        real_sign1, real_sign2 = newTorchSign(real_sign)

        err1 = criterion(real_sign1[k], pred_sign1[0]).item()
        err2 = criterion(real_sign2[k], pred_sign2[0]).item()

        all_errors_modulus1.append(err1)
        all_errors_modulus2.append(err2)

errors1 = np.array(all_errors_modulus1)
errors2 = np.array(all_errors_modulus2)

# Moyennes
mean1 = np.mean(errors1)
mean2 = np.mean(errors2)

# Écart-types
std1 = np.std(errors1, ddof=1)  # ddof=1 pour échantillon
std2 = np.std(errors2, ddof=1)

# Erreurs-types
stderr1 = std1 / np.sqrt(len(errors1))
stderr2 = std2 / np.sqrt(len(errors2))

# Intervalles de confiance à 95%
ci1 = stats.norm.interval(0.95, loc=mean1, scale=stderr1)
ci2 = stats.norm.interval(0.95, loc=mean2, scale=stderr2)

# === Affichage final ===
print("\n--- Résultats sur {:,} microstructures ---".format(n_samples))
print(f"Storage modulus (MAPE) : {mean1*100:.2f} ± {stderr1*100:.2f} % (IC 95%: {ci1[0]*100:.2f} – {ci1[1]*100:.2f} %)")
print(f"Loss modulus    (MAPE) : {mean2*100:.2f} ± {stderr2*100:.2f} % (IC 95%: {ci2[0]*100:.2f} – {ci2[1]*100:.2f} %)")

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random as rd
from traitement_image import image_processing
from utils import transform_batch, newTorchSign

n_samples = 2
latent_dim = 38

# Générer des exemples avec le générateur
batches = generate_batches_cond(data, n_samples)
i = rd.randrange(len(batches))
batch = batches[i]
cond = gan.generate_cond([sample[1] for sample in batch])
real_sign = np.expand_dims(cond, axis=-1)
random_indices = np.random.choice(cond.shape[0], 2, replace=False)
conditions_selected = cond[random_indices]
z = gan.generate_latent_points(latent_dim, n_samples)
input_model = layers.Concatenate()([z, conditions_selected])

generated_samples = generatorGP.predict_on_batch(input_model)

# Conversion des images générées (TensorFlow) en tenseur PyTorch
fake_images_torch = transform_batch(generated_samples)

# Prédiction de la signature avec le CNN-RNN
pred_sign1, pred_sign2 = modelCNN(fake_images_torch)
real_sign1, real_sign2 = newTorchSign(real_sign)

# Affichage des images et signatures (log scale pour les courbes)
X = np.logspace(-4, 2, real_sign1.shape[-1])  # fréquence logarithmique typique

for i, generated in enumerate(generated_samples):
    real_image = batch[i][0]
    
    # Affichage image
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(real_image, cmap='gray')
    plt.title("Real Image")
    
    plt.subplot(1, 2, 2)
    plt.imshow(generated, cmap='gray')
    plt.title("Generated Image")
    plt.show()
    
    # Affichage des courbes Storage / Loss avec échelle log
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # STORAGE modulus
    ax1.plot(X, torch.exp(real_sign1[i]).detach().numpy(), label="Real Sign 1")
    ax1.plot(X, torch.exp(pred_sign1[i]).detach().numpy(), label="Pred Sign 1")
    ax1.set_xscale('log')
    ax1.set_title(f"Storage modulus (MAPE = {100*criterion(real_sign1[i], pred_sign1[i]):.2f}%)")
    ax1.set_xlabel('Frequency (rad/s)')
    ax1.set_ylabel('Storage modulus (MPa)')
    ax1.legend()

    # LOSS modulus
    ax2.plot(X, torch.exp(real_sign2[i]).detach().numpy(), label="Real Sign 2")
    ax2.plot(X, torch.exp(pred_sign2[i]).detach().numpy(), label="Pred Sign 2")
    ax2.set_xscale('log')
    ax2.set_title(f"Loss modulus (MAPE = {100*criterion(real_sign2[i], pred_sign2[i]):.2f}%)")
    ax2.set_xlabel('Frequency (rad/s)')
    ax2.set_ylabel('Loss modulus (MPa)')
    ax2.legend()

    plt.show()

# Traitement global
image_processing(generated_samples, n_samples, real_sign, modelCNN)


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import random as rd 
from traitement_image import image_processing
from utils import transform_batch, newTorchSign
n_samples = 2
latent_dim=38
# Générer des exemples avec le générateur
batches = generate_batches_cond(data, n_samples)
i = rd.randrange(len(batches))
batch = batches[i]
cond = gan.generate_cond([sample[1] for sample in batch])
real_sign = np.expand_dims(cond,axis=-1)
random_indices = np.random.choice(cond.shape[0], 2, replace=False)
conditions_selected = cond[random_indices]
z = gan.generate_latent_points(latent_dim, n_samples)
input_model = layers.Concatenate()([z, conditions_selected])

X_fake = generatorGP.predict_on_batch(input_model)
plt.imshow(X_fake[0], cmap='gray')
plt.show()


from PIL import Image

X_fake_reshaped = (X_fake[0] * 255).astype(np.uint8).reshape(128, 128)
image = Image.fromarray(X_fake_reshaped, mode='L') 

output_path = os.path.join('/Users/Bader/Desktop/Mines 2A/Projet 2A/Images génerées', 'image1.png')
image.save(output_path)

import cv2
import numpy as np

image_path = r"/Users/Bader/Desktop/Mines 2A/Projet 2A/Images génerées/image1.png"

with open(image_path, "rb") as f:
    image_data = np.asarray(bytearray(f.read()), dtype=np.uint8)

#On décode l'image
imageenr = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
H=imageenr.shape[0]
W=imageenr.shape[1]
#print(imageenr)
print(imageenr.shape)
# #image = cv2.imread(image_gray)
gray = cv2.cvtColor(imageenr, cv2.COLOR_BGR2GRAY)


# #Appliquer un seuillage
# _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

# # ON évite les petits artefacts
# kernel = np.ones((3, 3), np.uint8)
# thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# # On trouve les contours des clusters
# contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Image sur laquelle on dessine les cercles
# output = np.zeros_like(image)

# # taille fixe des cercles
# fixed_radius = 5 
# fig, ax = plt.subplots()
# ax.imshow(output, cmap='gray')

# for cnt in contours:
#     M = cv2.moments(cnt)
#     if M["m00"] != 0:
#         cx = int(M["m10"] / M["m00"] )
#         cy = int(M["m01"] / M["m00"] )
        
#         # DOn dessine un cercle de taille fixe centré sur le cluster
#         #cv2.circle(output, (cx, cy), fixed_radius, (255, 255, 255), -1)
#         circle = plt.Circle((cx, cy), fixed_radius, color='white', fill=True, alpha=0.9)
#         ax.add_patch(circle)


# #plt.imshow(fig, cmap='gray')
# plt.axis()
# plt.show()
from scipy.ndimage import gaussian_filter, distance_transform_edt
from skimage.morphology import disk, binary_erosion
from skimage.segmentation import watershed 
from skimage.filters import threshold_otsu
from skimage.measure import label,regionprops
smoothed = gaussian_filter(gray,sigma=0.5)

tresh_value = threshold_otsu(smoothed)
binary = (smoothed > tresh_value).astype(np.uint8) * 255

kernel = np.ones((3, 3), np.uint8)
thresh = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)


kernel = disk(2)
binary_opened = binary_erosion(thresh, kernel)

distance=distance_transform_edt(binary_opened)
markers=label(distance>0.3*distance.max())
local_maxi = watershed(-distance,markers,mask=binary_opened)
binary_local_maxi = (local_maxi > 0).astype(np.uint8) * 255
fig,ax= plt.subplots(1,5,figsize=(15,5))
ax[0].imshow(imageenr,cmap='gray') ; ax[0].set_title("image originale")
ax[1].imshow(binary,cmap='gray') ; ax[1].set_title("image seuillée")
ax[2].imshow(thresh,cmap='gray') ; ax[2].set_title("ouverture morphologique")
ax[3].imshow(binary_opened,cmap='gray') ; ax[3].set_title("image clusters réduits")
ax[4].imshow(binary_local_maxi,cmap='nipy_spectral'); ax[4].set_title("image clusters séparés")

plt.show()
image_bw = Image.fromarray(binary_local_maxi, mode='L')
print(image)
output_path = os.path.join('/Users/Bader/Desktop/Mines 2A/Projet 2A/Images génerées', 'image2.png')
image_bw.save(output_path)

image_path = r"/Users/Bader/Desktop/Mines 2A/Projet 2A/Images génerées/image2.png"

with open(image_path, "rb") as f:
    image_data = np.asarray(bytearray(f.read()), dtype=np.uint8)

#On décode l'image
imageenr = cv2.imdecode(image_data, cv2.IMREAD_COLOR)

gray = cv2.cvtColor(imageenr, cv2.COLOR_BGR2GRAY)


#Appliquer un seuillage
_, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)

# ON évite les petits artefacts
kernel = np.ones((3, 3), np.uint8)
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# On trouve les contours des clusters
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Image sur laquelle on dessine les cercles
output = np.zeros_like(imageenr)

# taille fixe des cercles
fixed_radius =imageenr.shape[0]*3.98942280401433*10**(-2)


fig, ax = plt.subplots()
ax.imshow(output, cmap='gray')


for cnt in contours:
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"] )
        cy = int(M["m01"] / M["m00"] )
        
        # DOn dessine un cercle de taille fixe centré sur le cluster
        #cv2.circle(output, (cx, cy), fixed_radius, (255, 255, 255), -1)
        circle = plt.Circle((cx, cy), fixed_radius, color='white', fill=True, alpha=0.9)
        ax.add_patch(circle)

        translations = [
            (-W, 0),  # Gauche
            (W, 0),   # Droite
            (0, -H),  # Bas
            (0, H),   # Haut
            (-W, -H), # Coin Bas-Gauche
            (-W, H),  # Coin Haut-Gauche
            (W, -H),  # Coin Bas-Droite
            (W, H)    # Coin Haut-Droite
        ]

        for dx, dy in translations:
            shifted_circle = plt.Circle((cx + dx, cy + dy), fixed_radius, color='white', fill=True, alpha=0.9)
            ax.add_patch(shifted_circle)


plt.axis()
plt.show()

dpi = 25
figsize = ( imageenr.shape[0] / dpi, imageenr.shape[1] / dpi)

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.imshow(output, cmap='gray')


for cnt in contours:
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"] )
        cy = int(M["m01"] / M["m00"] )
        
        # DOn dessine un cercle de taille fixe centré sur le cluster
        #cv2.circle(output, (cx, cy), fixed_radius, (255, 255, 255), -1)
        circle = plt.Circle((cx, cy), fixed_radius, color='white', fill=True, alpha=0.9)
        ax.add_patch(circle)

        translations = [
            (-W, 0),  # Gauche
            (W, 0),   # Droite
            (0, -H),  # Bas
            (0, H),   # Haut
            (-W, -H), # Coin Bas-Gauche
            (-W, H),  # Coin Haut-Gauche
            (W, -H),  # Coin Bas-Droite
            (W, H)    # Coin Haut-Droite
        ]

        for dx, dy in translations:
            shifted_circle = plt.Circle((cx + dx, cy + dy), fixed_radius, color='white', fill=True, alpha=0.9)
            ax.add_patch(shifted_circle)
ax.set_position([0,0,1,1])

ax.set_xticks([])  
ax.set_yticks([])  
ax.set_frame_on(False)  
ax.set_facecolor('black')

# # Calculer la taille de la figure en pouces (445 pixels / 100 DPI)
# dpi = 100
# figsize = ( imageenr.shape[0] / dpi, imageenr.shape[1] / dpi)

# # Créer une figure de la taille de l'image
# fig = plt.figure(figsize=figsize, dpi=dpi)
# ax = fig.add_axes([0, 0, 1, 1])  # Axes qui remplissent toute la figure
# ax.set_axis_off()  # Désactive les axes
#plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Supprime les marges


fig.canvas.draw()


image_array = np.array(fig.canvas.renderer.buffer_rgba())  # Récupération en RGBA
plt.close(fig)  # Fermer la figure pour libérer la mémoire
print(image_array.shape)

# Conversion en niveaux de gris et seuillage
gray_image = np.mean(image_array[:H, :W, :3], axis=2)  # Convertir en niveaux de gris
binary_array = gray_image > 100  # Seuil pour obtenir une matrice de 0 et 1
print(binary_array.shape)
# Affichage du résultat
plt.figure()
plt.imshow(binary_array, cmap='gray')
plt.title("Image transformée en binaire")
plt.show()
# array = np.array(output)
# plt.imshow(array, cmap='gray')
# plt.show()

# cv2.imshow("Clusters remplacés par cercles fixes", output)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



# import cv2 
# import numpy as np 
# from skimage import morphology 
# from skimage.segmentation import watershed 
# from skimage.feature import peak_local_max 
# from scipy import ndimage 
# from skimage.transform import resize

# # # Charger l'image en niveaux de gris 
# # image = cv2.imread('image.png', cv2.IMREAD_GRAYSCALE) 

# # Appliquer un seuillage binaire
# _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# # Suppression du bruit avec une ouverture morphologique
# kernel = np.ones((3,3), np.uint8)
# opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

# # Déterminer l’arrière-plan (zone sûre)
# sure_bg = cv2.dilate(opening, kernel, iterations=3)

# # Déterminer l’avant-plan (objets à segmenter)
# dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
# _, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)

# # Trouver les marqueurs
# sure_fg = np.uint8(sure_fg)
# unknown = cv2.subtract(sure_bg, sure_fg)
# _, markers = cv2.connectedComponents(sure_fg)

# # Ajouter 1 à tous les marqueurs pour éviter l'étiquette 0
# markers = markers + 1

# # Les pixels inconnus sont marqués en 0
# markers[unknown == 255] = 0

# # Appliquer Watershed
# cv2.watershed(imageenr, markers)

# # Colorer les contours en rouge
# imageenr[markers == -1] = [0, 0, 255]

# # Afficher l’image segmentée
# plt.figure(figsize=(8,8))
# plt.imshow(cv2.cvtColor(imageenr, cv2.COLOR_BGR2RGB))
# plt.title("Segmentation Watershed")
# plt.show()

# # Étape 1: Filtrage Gaussien et Seuillage d'Otsu 
# blurred = cv2.GaussianBlur(gray, (5, 5), 0.5) 
# _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU) 

# # Étape 2: Opération Morphologique d'Ouverture 
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4)) 
# opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel) 

# # Étape 3: Transformation de Distance Euclidienne et Ligne de Partage des Eaux 
# distance = ndimage.distance_transform_edt(opened) 
# # print(distance.shape)
# # local_maxi = peak_local_max(distance, footprint=np.ones((4, 4)), labels=opened) 
# # print(local_maxi.shape)
# # markers = ndimage.label(local_maxi)[0] 
# # print(markers.shape)
# # print(opened.shape)
# # if markers.shape != opened.shape:
# #     # Resize markers to match the shape of opened
# #     markers = resize(markers, opened.shape, order=0, mode='constant', cval=0, clip=True)
# markers = ndimage.label(opened)[0]
# labels = watershed(-distance, markers, mask=opened) 

# # Afficher les résultats 
# #cv2.imshow('Original', imageenr)
# cv2.imshow('Binary', binary)
# cv2.imshow('Opened', opened)
# cv2.imshow('Watershed', labels.astype(np.uint8) * 255)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# gray = cv2.cvtColor(binary, cv2.COLOR_BGR2GRAY)


# #Appliquer un seuillage
# _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# # Appliquer une ouverture morphologique pour éviter les petits artefacts
# kernel = np.ones((5, 5), np.uint8)
# thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

# # Trouver les contours des clusters
# contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Créer une image noire pour dessiner les cercles
# output = np.zeros_like(image)

# # Définir la taille fixe des cercles
# fixed_radius = 5  # Ajuste selon ton besoin

# # Parcourir chaque cluster détecté
# for cnt in contours:
#     # Trouver le centre de gravité du cluster
#     M = cv2.moments(cnt)
#     if M["m00"] != 0:
#         cx = int(M["m10"] / M["m00"] )
#         cy = int(M["m01"] / M["m00"] )
#         # Dessiner un cercle de taille fixe centré sur le cluster
#         cv2.circle(output, (cx, cy), fixed_radius, (255, 255, 255), -1)

# # Afficher le résultat
# cv2.imshow("Clusters remplacés par cercles fixes", output)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()